In [2]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.sans-serif"] = "Arial"

In [3]:
#survival data is loaded
meta = pd.read_csv('OV.Survival.final.csv')
meta_columns = ['sample', 'days', 'stats']
meta.columns = meta_columns
meta

,sample,days,stats
0,TCGA-04-1357,9999,0
1,TCGA-04-1360,9999,0
2,TCGA-59-2348,5481,0
3,TCGA-13-0886,4665,0
4,TCGA-24-2020,4624,1
...,...,...,...
575,TCGA-24-2262,11,1
576,TCGA-23-1107,9,1
577,TCGA-25-1631,9,1
578,TCGA-30-1857,8,1


In [11]:
hrd_test = pd.read_csv('HRD_SCORE.txt', sep = '\t')
columns = ['sample', 'NtAI', 'LST', 'HRD-LOH', 'Nmut', 'FLOH', 'wGII', 'Ploidy']
hrd_test.columns = columns 
hrd_test

,sample,NtAI,LST,HRD-LOH,Nmut,FLOH,wGII,Ploidy
0,TCGA-BL-A0C8,14,13,7,162.0,0.0905,0.3885,3.2987
1,TCGA-C4-A0EZ,9,11,4,202.0,0.0806,0.4785,3.4094
2,TCGA-C4-A0F7,25,33,20,84.0,0.4648,0.6606,2.7365
3,TCGA-BL-A13J,26,11,13,135.0,0.3532,0.7270,4.3895
4,TCGA-CU-A0YN,19,7,12,175.0,0.2918,0.5793,2.2012
...,...,...,...,...,...,...,...,...
5366,TCGA-DI-A1NN,22,14,5,73.0,0.1124,0.6301,3.3878
5367,TCGA-EC-A1QX,3,3,1,NaN,0.0333,0.0441,2.0652
5368,TCGA-EC-A24G,0,0,0,NaN,0.0096,0.0047,2.0004
5369,TCGA-BG-A2AE,7,7,5,68.0,0.1075,0.2327,2.1482


In [12]:
sample_list = meta['sample'].to_list()
hrd_test = hrd_test[hrd_test['sample'].isin(sample_list)]
hrd_test

,sample,NtAI,LST,HRD-LOH,Nmut,FLOH,wGII,Ploidy
3266,TCGA-04-1519,7,3,6,2.0,0.3102,0.3724,1.7136
3267,TCGA-36-1578,31,34,13,44.0,0.4177,0.6221,2.5617
3268,TCGA-09-2043,30,26,25,NaN,0.5045,0.5211,2.2260
3269,TCGA-59-2349,3,2,0,NaN,0.0126,0.0354,2.0392
3270,TCGA-24-2293,29,16,20,35.0,0.4424,0.5616,2.7229
...,...,...,...,...,...,...,...,...
3773,TCGA-61-2109,30,34,22,1.0,0.4302,0.6284,3.3665
3774,TCGA-61-2110,6,4,2,19.0,0.3278,0.4333,1.7971
3775,TCGA-61-1721,0,0,0,NaN,0.0080,0.0006,2.0001
3776,TCGA-25-2398,26,11,8,30.0,0.1610,0.6620,4.1951


### HRD1 : survival analysis
- 'hrd_survival' : survival stats of 510 patients
- 'hrd test' : HRD score of 510 patients

In [49]:
samplelist = list(hrd_test['sample'])
len(samplelist)


analysis1_p = []; analysis1_prog = []
analysis2_p = []; analysis2_prog = []
analysis3_p = []; analysis3_prog = []
analysis4_p = []; analysis4_prog = []

nsample = 1
mysample=[]

df = pd.concat([hrd_survival[['sample', 'stats', 'days']], hrd_test['HRD']])
df

,0,days,sample,stats
0,NaN,9999.0,TCGA-04-1357,0.0
1,NaN,9999.0,TCGA-04-1360,0.0
3,NaN,4665.0,TCGA-13-0886,0.0
4,NaN,4624.0,TCGA-24-2020,1.0
5,NaN,4424.0,TCGA-13-0890,0.0
...,...,...,...,...
3773,86.0,NaN,NaN,NaN
3774,12.0,NaN,NaN,NaN
3775,0.0,NaN,NaN,NaN
3776,45.0,NaN,NaN,NaN


In [42]:


for mysample in samplelist:  

    #df['log2expression'] = pd.Series([np.log2(i+1) for i in df['expression']], index=df.index)

    # -------------------------- Survival analysis 1 --------------------------
    # High expression = above HRD score of 42, low expression = below HRD score of 42

    c1 = 42
    #print("Cut-off for high expression:",c1,"; Cut-off for low expression:", c2)
    kmf = KaplanMeierFitter()
    T = df['stats']
    E = df['censor']
    l1 = (df['HRD']>=c1)
    l2 = (df['HRD']<=c1)
    #print(sum(l1),"patients are in high expression group;",sum(l2), "patients are in low expression group")
    results = logrank_test(T[l1], T[l2], E[l1], E[l2], alpha=.95)
    analysis1_p.append(results.p_value)
    #print("P-value between low and high expression group:", results.p_value)
    if results.p_value<0.05:
        kmf.fit(T[l1], event_observed=E[l1], label="HRD > 42")
        median_high_group = kmf.median_survival_time_
        kmf.fit(T[l2], event_observed=E[l2], label="HRD < 42")
        median_low_group = kmf.median_survival_time_
        if median_high_group<median_low_group:
            analysis1_prog.append("unfavorable")
        else:
            analysis1_prog.append("favorable")
    else:
        analysis1_prog.append("non-significant")
    


  

UnboundLocalError: local variable 'survival_table' referenced before assignment

In [43]:
mysample

'TCGA-04-1519'

In [ ]:
    # -------------------------- Survival analysis 2 --------------------------
    # High expression = above median, low expression = below median
    c1 = np.quantile(hrd_test['HRD'], q = 0.25) 
    c2 = np.quantile(hrd_test['HRD'], q = 0.75) 

    #print("Cut-off for high expression: >=",c1)
    kmf = KaplanMeierFitter()
    T = df['stats']
    E = df['censor']
    l1 = (df['HRD']>c1)
    l2 = (df['HRD']<=c2)
    #print(sum(l1),"patients are in high expression group;",sum(l2), "patients are in low expression group")
    results = logrank_test(T[l1], T[l2], E[l1], E[l2], alpha=.95)
    analysis2_p.append(results.p_value)
    #print("P-value between low and high expression group:", results.p_value)
    if results.p_value<0.05:
        kmf.fit(T[l1], event_observed=E[l1], label="High-"+mysample)
        median_high_group = kmf.median_survival_time_
        kmf.fit(T[l2], event_observed=E[l2], label="low-"+mysample)
        median_low_group = kmf.median_survival_time_
        if median_high_group<median_low_group:
            analysis2_prog.append("unfavorable")
        else:
            analysis2_prog.append("favorable")
    else:
        analysis2_prog.append("non-significant")
    

In [ ]:
  # -------------------------- Survival analysis 3 --------------------------
    # Cox partial hazard fitting
    df2 = df.loc[:,['expression','OS','censor']]
    cph = CoxPHFitter()
    cph.fit(df2, duration_col='OS', event_col='censor', step_size=0.50)
    analysis4_p.append(cph.summary['p'].values[0])
    if cph.summary['p'].values[0]<0.05:
        if cph.summary['exp(coef)'].values[0]>1:
            analysis4_prog.append("unfavorable")
        else:
            analysis4_prog.append("favorable")
    else:
        analysis4_prog.append("non-significant")
        
    print(nsample, "Completed analysis for",nsample)
    nsample+=1

In [6]:
print(len(analysis1_p),len(analysis1_prog),len(analysis2_p),len(analysis2_prog),
     len(analysis3_p),len(analysis3_prog),len(analysis4_p),len(analysis4_prog))

188 188 188 188 188 188 188 188


In [7]:
data['Geneid']

0                  MSTRG.1
1                  MSTRG.2
2                MSTRG.151
3                MSTRG.176
4                MSTRG.212
               ...        
32287    ENSG00000228296.1
32288    ENSG00000223641.1
32289    ENSG00000228786.1
32290    ENSG00000240450.1
32291    ENSG00000231141.1
Name: Geneid, Length: 32292, dtype: object

In [19]:
final = pd.DataFrame({'Geneid':lnclist,'Gene_name':list(diff['Geneid']),
                      'Pvalue1':analysis1_p,'Prognosis1':analysis1_prog,
                      'Pvalue2':analysis2_p,'Prognosis2':analysis2_prog,
                      'Pvalue3':analysis3_p,'Prognosis3':analysis3_prog,
                      'Pvalue4':analysis4_p,'Prognosis4':analysis4_prog}, 
                     columns=['Geneid','Gene_name','Pvalue1','Prognosis1',
                              'Pvalue2','Prognosis2','Pvalue3','Prognosis3',
                              'Pvalue4','Prognosis4'])
final

,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4
0,MSTRG.388,MSTRG.388,0.665818,non-significant,0.857462,non-significant,0.241985,non-significant,0.687800,non-significant
1,MSTRG.1064,MSTRG.1064,0.076831,non-significant,0.285178,non-significant,0.016046,favorable,0.161146,non-significant
2,MSTRG.1065,MSTRG.1065,0.100247,non-significant,0.217918,non-significant,0.046359,favorable,0.253932,non-significant
3,MSTRG.6161,MSTRG.6161,0.310938,non-significant,0.705849,non-significant,0.003212,unfavorable,0.025569,unfavorable
4,MSTRG.12332,MSTRG.12332,0.811638,non-significant,0.418578,non-significant,0.066341,non-significant,0.373222,non-significant
...,...,...,...,...,...,...,...,...,...,...
183,ENSG00000232855.2,ENSG00000232855.2,0.761402,non-significant,0.925650,non-significant,0.278580,non-significant,0.925869,non-significant
184,ENSG00000225465.6,ENSG00000225465.6,0.419565,non-significant,0.049925,favorable,0.025260,favorable,0.458891,non-significant
185,ENSG00000260655.1,ENSG00000260655.1,0.015293,favorable,0.846979,non-significant,0.002695,favorable,0.077322,non-significant
186,ENSG00000260822.1,ENSG00000260822.1,0.250255,non-significant,0.104284,non-significant,0.030107,favorable,0.883972,non-significant


In [20]:
from collections import Counter
for i in ['Prognosis1','Prognosis2','Prognosis3','Prognosis4']:
    print(Counter(final[i]))

Counter({'non-significant': 162, 'favorable': 23, 'unfavorable': 3})
Counter({'non-significant': 165, 'favorable': 19, 'unfavorable': 4})
Counter({'non-significant': 110, 'favorable': 63, 'unfavorable': 15})
Counter({'non-significant': 183, 'unfavorable': 5})


In [21]:
final[(final['Prognosis1']=='unfavorable')&(final['Prognosis2']=="unfavorable")&
      (final['Prognosis3']=='unfavorable')&(final['Prognosis4']=='unfavorable')]


,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4


In [22]:
final[final['Pvalue3']<0.05/4700]

,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4
152,ENSG00000246283.2,ENSG00000246283.2,0.005094,favorable,0.000021,favorable,0.000002,favorable,0.058329,non-significant


In [23]:
final[final['Gene_name']=="LINC01614"]

,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4


In [24]:
final[final['Gene_name']=="H19"]

,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4


In [25]:
final[final['Gene_name']=="DUXAP8"]

,Geneid,Gene_name,Pvalue1,Prognosis1,Pvalue2,Prognosis2,Pvalue3,Prognosis3,Pvalue4,Prognosis4


In [26]:
final.to_csv('Survival_lncRNA_expression_discovery.txt', sep="\t", index=False)